# Aggregations & GROUP BY

**Use case:** Building a sales reporting dashboard.

This notebook covers the aggregation API -- quick totals, averages, and
grouped summaries. These are essential for dashboards, analytics, and any
feature that needs to summarize data rather than list individual records.

## Prerequisites

A SurrealDB Docker container must be running. See `00_setup.ipynb` for details.

In [ ]:
# -- Setup -----------------------------------------------------------------
import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)
print("Connection configured.")

## Define the Order Model and Seed Data

We'll create a batch of orders with different statuses, amounts, and
categories to demonstrate aggregation queries.

In [ ]:
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict


class Order(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="orders")

    id: str | None = None
    customer_name: str
    amount: float
    status: str = "pending"    # pending, completed, cancelled
    category: str = "general"


# Seed sample orders
orders_data = [
    Order(customer_name="Alice", amount=150.00, status="completed",  category="electronics"),
    Order(customer_name="Bob",   amount=45.50,  status="completed",  category="books"),
    Order(customer_name="Alice", amount=320.00, status="completed",  category="electronics"),
    Order(customer_name="Charlie", amount=89.99, status="pending",   category="clothing"),
    Order(customer_name="Diana", amount=210.00, status="completed",  category="electronics"),
    Order(customer_name="Bob",   amount=12.99,  status="cancelled",  category="books"),
    Order(customer_name="Eve",   amount=75.00,  status="pending",    category="clothing"),
    Order(customer_name="Alice", amount=55.00,  status="completed",  category="books"),
    Order(customer_name="Frank", amount=430.00, status="completed",  category="electronics"),
    Order(customer_name="Diana", amount=18.50,  status="cancelled",  category="general"),
]

for order in orders_data:
    await order.save()

print(f"Seeded {len(orders_data)} orders.")

## Quick Aggregations

The QuerySet provides shortcut methods for the most common aggregations.
These return a single scalar value and run efficiently on the server side.

Use these when you need a single number -- the total count, the sum of a
column, or the average price.

In [ ]:
# Count all orders
total_count = await Order.objects().count()
print(f"Total orders: {total_count}")

# Sum of all order amounts
total_revenue = await Order.objects().sum("amount")
print(f"Total revenue: ${total_revenue:.2f}")

# Average order amount
avg_amount = await Order.objects().avg("amount")
print(f"Average order: ${avg_amount:.2f}")

# Minimum and maximum order amounts
min_amount = await Order.objects().min("amount")
max_amount = await Order.objects().max("amount")
print(f"Smallest order: ${min_amount:.2f}")
print(f"Largest order:  ${max_amount:.2f}")

In [ ]:
# Aggregations can be combined with filters.
# For example, total revenue from completed orders only:
completed_revenue = await Order.objects().filter(status="completed").sum("amount")
print(f"Revenue (completed only): ${completed_revenue:.2f}")

# Count of pending orders
pending_count = await Order.objects().filter(status="pending").count()
print(f"Pending orders: {pending_count}")

## GROUP BY with `values()` and `annotate()`

For reporting dashboards, you often need *grouped* summaries -- for example,
total sales per status, or average order per category.

The pattern is:

```python
await Model.objects().values("group_field").annotate(
    alias=AggregationFunction("field"),
).exec()
```

- **`values("field")`** -- Sets the GROUP BY column(s).
- **`annotate(alias=...)`** -- Adds computed aggregation columns.

Available aggregation classes: `Count`, `Sum`, `Avg`, `Min`, `Max`.

In [ ]:
from src.surreal_orm import Count, Sum, Avg

# Revenue breakdown by order status
status_stats = await Order.objects().values("status").annotate(
    count=Count(),
    total=Sum("amount"),
    avg_amount=Avg("amount"),
).exec()

print("Revenue by status:")
print(f"{'Status':<12} {'Count':>6} {'Total':>10} {'Average':>10}")
print("-" * 42)
for row in status_stats:
    print(f"{row['status']:<12} {row['count']:>6} ${row['total']:>9.2f} ${row['avg_amount']:>9.2f}")

In [ ]:
# Sales by category -- only completed orders
category_stats = await Order.objects().filter(status="completed").values("category").annotate(
    count=Count(),
    total=Sum("amount"),
).exec()

print("Completed sales by category:")
print(f"{'Category':<15} {'Count':>6} {'Total':>10}")
print("-" * 35)
for row in category_stats:
    print(f"{row['category']:<15} {row['count']:>6} ${row['total']:>9.2f}")

## Putting It Together: A Mini Dashboard

Here's how you might build a summary for a sales dashboard, combining
scalar aggregations and grouped queries.

In [ ]:
from src.surreal_orm import Min, Max

# Top-line metrics
total = await Order.objects().count()
revenue = await Order.objects().filter(status="completed").sum("amount")
avg_order = await Order.objects().filter(status="completed").avg("amount")
biggest = await Order.objects().max("amount")
smallest = await Order.objects().min("amount")

print("=" * 45)
print("          SALES DASHBOARD SUMMARY")
print("=" * 45)
print(f"  Total orders:        {total}")
print(f"  Completed revenue:   ${revenue:.2f}")
print(f"  Avg completed order: ${avg_order:.2f}")
print(f"  Largest order:       ${biggest:.2f}")
print(f"  Smallest order:      ${smallest:.2f}")
print("=" * 45)

## Cleanup

Remove the orders table so the database is clean for other notebooks.

In [ ]:
client = await SurrealDBConnectionManager.get_client()
await client.query("REMOVE TABLE orders;")
print("Cleanup complete -- orders table removed.")